### Structural Estimation Q4

First, we once again install some required packages and load the data.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
%matplotlib inline

ms = pd.read_csv("MacroSeries.txt", header = None, names = ["c","k","w","r"])

Let us also define some functions to back out a series for z, as well as the four empirical moments.

In [16]:
def z4(r, k, alpha):
    ezt = r/((alpha)*k**(alpha -1))
    return np.log(ezt)

def model_moments(zvec, kvec, cvec, wvec, alpha, rho, mu, beta):
    moments = np.zeros(4)
    moments[0] = np.mean(zvec[1:] - rho*(zvec[:-1]) - (1-rho)*mu)
    moments[1] = np.mean((zvec[1:] - rho*(zvec[:-1]) - (1 - rho)*mu)*zvec[:-1])
    moments[2] = np.mean(beta*alpha*np.exp(zvec[1:])*(kvec[1:]**(alpha -1))*(cvec[:-1]/cvec[1:]) - 1)
    moments[3] = np.mean((beta*alpha*np.exp(zvec[1:])*(kvec[1:]**(alpha-1))*(cvec[:-1]/cvec[1:]) - 1)*wvec[:-1])
    return moments

def crit(params, *args):
    #Note here that since we assume each of these moments should be 0, the error vector is simply the vector of all moments.
    alpha, rho, mu, beta = params
    rvec, kvec, cvec, wvec, W = args
    zvec = z4(rvec, kvec, alpha)
    err = abs(model_moments(zvec, kvec, cvec, wvec, alpha, rho, mu, beta))
    crit_val = np.dot(np.dot(err.T, W), err)
    return crit_val
    

In [28]:
params = (0.45, 0.7, 11, 0.9)
args = (ms['r'], ms['k'], ms['c'], ms['w'], np.eye(4))

results = opt.minimize(crit, params, args, method='Nelder-Mead',
                       bounds=((0,1), (-1,1), (None, None),(0,1)))

gmm_alpha , gmm_rho, gmm_mu, gmm_beta = results.x

print("GMM Estimate for Alpha = ", gmm_alpha, "\nGMM Estimate for Rho = ", gmm_rho, 
     "\nGMM Estimate for Mu = ", gmm_mu, "\nGMM Estimate for Beta = ", gmm_beta,
      "\nCriterion Value = ", results.fun )

C:\Users\Kenneth\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:394: RuntimeWarning: Method Nelder-Mead cannot handle constraints nor bounds.
  RuntimeWarning)


GMM Estimate for Alpha =  0.443400129138 
GMM Estimate for Rho =  0.746999257765 
GMM Estimate for Mu =  10.4479730672 
GMM Estimate for Beta =  0.976572309367 
Criterion Value =  4.082987589


We note that these values vary slightly from the ones computed in the MLE exercise, namely a slightly larger $\alpha$ and a slightly larger $\mu$. However, the general size of $\alpha$, the capital share of inputs remains close to 0.45, and similarly the general estimate for the persistence of the shock ($\rho$) is also close to 0.7. As a check, we also note that the estimated value of $\beta = 0.976$ is close to what we usually assume in solving these models.